In [ ]:
import networkx as nx
import numpy as np
from scipy.optimize import linprog
from copy import deepcopy
from math import factorial, exp


Step 1: Imports and basic setup done.


In [ ]:
import logging
logger = logging.getLogger()

In [88]:
%run "attack_graph_MARA.ipynb"

## Add Virtual starting node

In [89]:
def find_and_add_entry_node(graph):
    # Find all nodes with no incoming edges
    roots = [n for n, deg in graph.in_degree() if deg == 0]
    #print("DEBUG: Roots before adding entry node:", roots)
    
    if len(roots) > 1:
        # Create a virtual entry node
        entry = "attacker_entry_node"
        graph.add_node(entry)
        
        # Connect the entry node to all roots
        for r in roots:
            graph.add_edge(entry, r, weight=1)
        
        # print("DEBUG: Multiple roots found. Created entry node:", entry)
        # print("DEBUG: Edges from entry node to roots:")
        # for r in roots:
        #     print(f"  {entry} -> {r}")
        # return entry, graph
    else:
        # If single root exists, that is our entry
        entry = roots[0]
        #print("DEBUG: Only one root found. Entry node is:", entry)
        return entry, graph


In [ ]:
def process_target_nodes(graph):
    # In R:
    # jointVertex <- gorder(attack_graph) - length(target_list) + 1
    # vertexNo[, target_list] <- jointVertex
    # vertexNo[vertexNo == 0] <- 1:(jointVertex - 1)
    # attack_graph <- contract.vertices(attack_graph, mapping = vertexNo)

    # Step 1: Identify target nodes (no outgoing edges)
    targets = [n for n, deg in graph.out_degree() if deg == 0]
    if len(targets) <= 1:
        # If there is 0 or 1 target, no merging needed
        return graph

    # Number of original nodes
    original_count = graph.number_of_nodes()
    # Same formula as in R code
    joint_vertex_num = original_count - len(targets) + 1

    # We will mimic exactly the logic from R:
    # First, get a topological order of the original graph
    topo_nodes = list(nx.topological_sort(graph))

    # Create a mapping array: For each node in topo order, assign a "vertexNo".
    # Initialize everything to 0
    vertexNo = {node: 0 for node in topo_nodes}

    # Set all targets to joint_vertex_num
    for t in targets:
        vertexNo[t] = joint_vertex_num

    # Now fill the zeros with consecutive integers from 1 to joint_vertex_num-1
    # but maintain the topological order
    fill_value = 1
    for node in topo_nodes:
        if vertexNo[node] == 0:
            vertexNo[node] = fill_value
            fill_value += 1

    # Now we have a mapping of each original node to an integer ID.
    # Nodes assigned the same integer ID should be contracted into one node.
    # Example: If multiple targets got joint_vertex_num, they form one merged node.

    # Create a mapping from the integer IDs to sets of original nodes
    block_map = {}
    for node, grp_id in vertexNo.items():
        if grp_id not in block_map:
            block_map[grp_id] = set()
        block_map[grp_id].add(node)

    # Build a new contracted graph
    merged_graph = nx.DiGraph()

    # Add nodes (each group_id becomes one node in merged_graph)
    for grp_id in block_map:
        merged_graph.add_node(grp_id)

    # For edges: if u->v in the old graph, map them to their groups:
    # If they map to the same group, skip (self-loop)
    # Otherwise, add edge between the groups
    for u, v, data in graph.edges(data=True):
        u_id = vertexNo[u]
        v_id = vertexNo[v]
        if u_id != v_id:
            # Keep the weight as is
            w = data.get('weight', 1)
            if not merged_graph.has_edge(u_id, v_id):
                merged_graph.add_edge(u_id, v_id, weight=w)

    return merged_graph

In [91]:
# Prepare the graph
work_graph = deepcopy(attack_graph)
entry_node, work_graph = find_and_add_entry_node(work_graph)

# print("DEBUG: Current graph nodes after handling entry node:", list(work_graph.nodes()))
# print("DEBUG: Current graph edges after handling entry node:")
# for u,v in work_graph.edges():
#     print(f"  {u} -> {v}")

work_graph = process_target_nodes(work_graph)

# print("DEBUG: Final graph after processing targets:")
# print("Nodes:", list(work_graph.nodes()))
# print("Edges:")
# for u,v in work_graph.edges():
#     print(f"  {u} -> {v}")

# new_targets = [n for n, deg in work_graph.out_degree() if deg == 0]
# print("DEBUG: New targets after merging:", new_targets)

## Next part

In [ ]:
def generate_game_elements(graph, entry_node):
    """
    Generate game elements after we have the final graph (with single entry and single target).
    """
    target_list = [n for n,d in graph.out_degree() if d == 0]
    if len(target_list) != 1:
        print("WARNING: Expected exactly one target node after contraction. Found:", target_list)
    
    routes = list(nx.all_simple_paths(graph, entry_node, target_list[0]))
    V = sorted(list(set([node for path in routes for node in path])))
    topo_all = list(nx.topological_sort(graph))
    node_order = [n for n in topo_all if n in V]
    
    excluded = {entry_node} | set(target_list)
    if entry_node == "attacker_entry_node":
        roots = [r for r, deg in graph.in_degree() if deg == 0 and r != entry_node]
        excluded.update(roots)
    
    as1 = [n for n in V if n not in excluded]
    as2 = routes
    
    return routes, V, as1, as2, target_list, node_order

In [93]:
def setup_game_parameters(V, routes, entry_node, target_list):
    adv_list = [n for n in V if n not in [entry_node] + target_list]
    if len(adv_list) == 0:
        print("WARNING: No adversary intermediate locations found. Check graph structure.")
    
    theta = {loc: 1/len(adv_list) for loc in adv_list}
    m = len(routes)
    return adv_list, theta, m

In [94]:
def lossDistribution(U):
    U = U / np.sum(U)
    support = np.arange(1, len(U)+1)
    dpdf = U
    cdf = np.cumsum(U)
    tail = 1 - cdf + U
    return {
        'support': support,
        'dpdf': dpdf,
        'cdf': cdf,
        'tail': tail
    }

In [95]:
def get_payoff_matrix(G, weights=None):
    """
    Compute a single aggregated payoff matrix from the MOSG object G, with debug statements.
    We assume:
    - G['nDefenses'] = n
    - G['nAttacks'] = m
    - G['dim'] = d
    - G['losses'] = list of length n*m*d loss distributions
    - G['targetNode'] = the actual target node's name or index
    - G['node_order'] = topological order of nodes
    - Each ld in G['losses'] is discrete and maps 1,...,len(V) to nodes in node_order
    
    We'll find the index of the target node in node_order and use ld['dpdf'][target_index] 
    to retrieve the attacker's success probability.
    """
    print("DEBUG: Entering get_payoff_matrix")

    n = G['nDefenses']
    m = G['nAttacks']
    d = G['dim']

    if weights is None:
        weights = np.full(d, 1.0/d)
    else:
        weights = np.array(weights, dtype=float)
        weights = weights / np.sum(weights)

    # Find target node index
    # We assume there's a single target. If multiple, adjust accordingly.
    if 'goalDescriptions' in G and len(G['goalDescriptions']) == 1:
        # For single goal, we have one target node in target_list
        # G should have 'target_list' and 'node_order'
        # If not, ensure these are stored in G or adapt accordingly.
        if 'target_list' not in G or 'node_order' not in G:
            raise ValueError("G must have 'target_list' and 'node_order' keys.")
        
        target_node = G['target_list'][0]
        # node_order is a list of node IDs in topological order
        # Find index in node_order
        if target_node not in G['node_order']:
            raise ValueError("Target node not found in node_order.")
        target_index = G['node_order'].index(target_node)
    else:
        # If no 'goalDescriptions' or multiple, we assume single target is last node
        # But better to raise an error if not sure
        raise ValueError("Ambiguous target node indexing. Provide 'target_list' and 'node_order' in G.")

    payoff_matrix = np.zeros((n, m))
    
    # Debug prints to verify indexing
    print(f"DEBUG: n={n}, m={m}, d={d}, target_index={target_index}, weights={weights}")

    for i in range(n):
        for j in range(m):
            val = 0.0
            for p in range(d):
                idx = i*m*d + j*d + p
                ld = G['losses'][idx]
                # attacker success probability is dpdf at target_index
                if target_index >= len(ld['dpdf']):
                    raise ValueError("target_index out of range for dpdf.")
                attacker_success_prob = ld['dpdf'][target_index]
                val += weights[p] * attacker_success_prob
            payoff_matrix[i,j] = val
            # Debug each cell
            print(f"DEBUG: payoff_matrix[{i},{j}]={payoff_matrix[i,j]}")

    print("DEBUG: Finished get_payoff_matrix")
    return payoff_matrix

In [96]:
def calculate_payoff_distribution(graph, as1, as2, V, adv_list, theta, random_steps_fn, 
                                  attack_rate, defense_rate, node_order):
    payoffs = []
    for i in as1:
        for path in as2:
            U = np.zeros(len(V))
            for avatar in adv_list:
                L = np.zeros(len(V))
                if avatar in path:
                    start_idx = path.index(avatar)
                    route = path[start_idx:]
                    pdf_d = random_steps_fn(route, attack_rate, defense_rate)
                    
                    if i in route:
                        cutPoint = route.index(i) + 1
                    else:
                        cutPoint = len(route)
                    
                    if np.sum(pdf_d[:cutPoint]) == 0:
                        payoffDistr = np.zeros(cutPoint)
                        payoffDistr[-1] = 1.0
                    else:
                        payoffDistr = pdf_d[:cutPoint] / np.sum(pdf_d[:cutPoint])
                    
                    for idx, node in enumerate(route[:cutPoint]):
                        L[V.index(node)] = payoffDistr[idx]
                else:
                    L[V.index(avatar)] = 1.0
                
                U += theta[avatar] * L
            
            if np.sum(U) == 0:
                U = np.full_like(U, 1e-7)
            else:
                U /= np.sum(U)
            
            node_positions = [V.index(n) for n in node_order]
            U = U[node_positions]
            U[U == 0] = 1e-7
            ld = lossDistribution(U)
            payoffs.append(ld)
    return payoffs

In [97]:
def solve_stage(Ui, v_prev=None, tol=0.0):
    """
    Solve one stage of the lexicographic optimization LP.
    Ui: n-by-m matrix
    """
    n, m = Ui.shape
    c = np.zeros(n+1)
    c[0] = 1.0

    A_ub = np.zeros((m, n+1))
    b_ub = np.zeros(m)
    for j in range(m):
        A_ub[j,0] = -1.0
        for i in range(n):
            A_ub[j,i+1] = Ui[i,j]

    A_eq = np.zeros((1, n+1))
    A_eq[0,1:] = 1.0
    b_eq = np.array([1.0])

    if v_prev is not None:
        A_ub = np.vstack([A_ub, np.zeros((1,n+1))])
        A_ub[-1,0] = 1.0
        b_ub = np.append(b_ub, v_prev + tol)

    bounds = [(0,None)]*(n+1)
    res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')
    if not res.success:
        raise ValueError("LP solver failed in solve_stage: " + res.message)

    v_opt = res.x[0]
    x_def = res.x[1:]
    return v_opt, x_def


In [98]:
def lexicographic_optimization(U_matrices, tol=0.0):
    """
    Perform lexicographic optimization over multiple stages.
    U_matrices: list of Ui arrays
    """
    v_values = []
    x_def_final = None
    v_prev = None

    for Ui in U_matrices:
        v_opt, x_def = solve_stage(Ui, v_prev=v_prev, tol=tol)
        v_values.append(v_opt)
        x_def_final = x_def
        v_prev = v_opt

    return v_values, x_def_final

In [99]:
def attacker_solve_stage(Ui, defender_v, w_prev=None, tol=0.0):
    """
    Solve one stage of the attacker's lexicographic optimization.
    Now we add both upper and lower bounds on w:
    w ≤ defender_v + tol (already present)
    w ≥ defender_v - tol (new)
    """
    n, m = Ui.shape
    # Variables: w,y_1,...,y_m
    c = np.zeros(m+1)
    c[0] = -1.0  # minimize -w => maximize w

    # w - ∑(Ui[i,j]*y_j) ≥ 0 => -w + ∑(Ui[i,j]*y_j) ≤ 0
    A_ub = np.zeros((n, m+1))
    b_ub = np.zeros(n)
    for i in range(n):
        A_ub[i,0] = -1.0
        for j in range(m):
            A_ub[i,j+1] = Ui[i,j]

    # sum_j y_j = 1
    A_eq = np.zeros((1, m+1))
    A_eq[0,1:] = 1.0
    b_eq = np.array([1.0])

    # Add upper bound: w ≤ defender_v + tol
    A_ub = np.vstack([A_ub, np.zeros((1,m+1))])
    A_ub[-1,0] = 1.0
    b_ub = np.append(b_ub, defender_v + tol)

    # Add lower bound: w ≥ defender_v - tol => -w ≤ -(defender_v - tol)
    # Add another row: -1*w ≤ -(defender_v - tol)
    A_ub = np.vstack([A_ub, np.zeros((1,m+1))])
    A_ub[-1,0] = -1.0
    b_ub = np.append(b_ub, -(defender_v - tol))

    # If w_prev is given, also ensure w ≤ w_prev + tol for lexicographic refinement
    if w_prev is not None:
        A_ub = np.vstack([A_ub, np.zeros((1,m+1))])
        A_ub[-1,0] = 1.0
        b_ub = np.append(b_ub, w_prev + tol)

    bounds = [(0,None)]*(m+1)
    res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')
    if not res.success:
        raise ValueError("LP solver failed in attacker_solve_stage: " + res.message)

    w_opt = res.x[0]
    y_att = res.x[1:]
    return w_opt, y_att


def attacker_lexicographic_optimization(U_matrices_for_attacker, defender_v, tol=0.0):
    """
    Perform lexicographic optimization for the attacker.
    U_matrices_for_attacker: list of Ui arrays
    defender_v: final defender equilibrium value to ensure boundedness
    """
    w_values = []
    y_att_final = None
    w_prev = None

    for Ua in U_matrices_for_attacker:
        w_opt, y_att = attacker_solve_stage(Ua, defender_v=defender_v, w_prev=w_prev, tol=tol)
        w_values.append(w_opt)
        y_att_final = y_att
        w_prev = w_opt

    return w_values, y_att_final

In [ ]:
def run_game(graph, attack_rate_list, defense_rate_list, random_steps_fn):
    entry_node_candidates = [n for n,deg in graph.in_degree() if deg==0]
    if len(entry_node_candidates) != 1:
        print("WARNING: Not a single entry node found.")
    entry_node = entry_node_candidates[0]

    routes, V, as1, as2, target_list, node_order = generate_game_elements(graph, entry_node)
    adv_list, theta, m = setup_game_parameters(V, routes, entry_node, target_list)

    # attacker actions = as2 (list of routes)
    # defender actions = as1 (list of nodes)
    n = len(as1)
    m = len(as2)
    d = 1  # Since we merged targets, we assume a single goal

    if not defense_rate_list:
        defense_rate_list = [0]
    if not attack_rate_list:
        attack_rate_list = [0]

    for defenseRate in defense_rate_list:
        for attackRate in attack_rate_list:
            logger.info("\n++++++++++++++++++++++++++++++++")
            logger.info(f"attack rate =  {attackRate} , defense rate =  {defenseRate} \n")
            logger.info("\tequilibrium for multiobjective security game (MOSG)\n")

            payoffs = calculate_payoff_distribution(
                graph, as1, as2, V, adv_list, theta, random_steps_fn, attackRate, defenseRate, node_order
            )

            # Construct G:
            G = {
                'nDefenses': n,
                'nAttacks': m,
                'dim': d,
                'losses': payoffs,
                'defensesDescriptions': [str(a) for a in as1],
                'attacksDescriptions': [str(r) for r in as2],
                'goalDescriptions': ['1'],  # single goal
                'maximumLoss': len(V),
                'target_list': target_list,
                'node_order': node_order
            }

            payoff_matrix = get_payoff_matrix(G)
            U_matrices = [payoff_matrix]

            # Compute defender equilibrium
            v_values, x_def = lexicographic_optimization(U_matrices, tol=1e-5)
            optimal_defense = dict(zip(G['defensesDescriptions'], x_def))
            eq = {
                'optimalDefense': optimal_defense,
                'assurances': {'1': {'dpdf': np.array([v_values[-1]])}},
                'attackerSuccess': v_values[-1]
            }

            # Log the optimal defense strategy
            logger.info("optimal defense strategy:")
            logger.info("         prob.")
            for node, prob in sorted(eq['optimalDefense'].items(), key=lambda x: str(x[0])):
                logger.info(f"{node} {prob:.6e}")


            # Compute attacker worst-case strategy

            defender_v = v_values[-1]
            attacker_U = [payoff_matrix.T]
            w_values, y_att = attacker_lexicographic_optimization(attacker_U, defender_v, tol=1e-5)

            # Log worst-case attack strategies once
            logger.info("\nworst case attack strategies per goal:")
            logger.info("          1")
            for idx, val in enumerate(y_att, start=1):
                logger.info(f"{idx} {val:.7f}")
            logger.info(f"[1] {y_att[-1]:.3f}")


## Test method in Case I want to run this notebook here

In [101]:
def test():
    print(f'We are in the main file, this will not be executed when imported in experiment_1.')
    # Define parameters
    attack_rate_list = [2]
    defense_rate_list = [0]

    def random_steps(route, attack_rate=None, defense_rate=None):
        length = len(route)
        if attack_rate is None:
            attack_rate = 2
        x = np.arange(length)
        pmf = (np.power(attack_rate, x) * np.exp(-attack_rate)) / np.vectorize(factorial)(x)
        pmf = pmf / pmf.sum()
        return pmf

    # Now run the game on work_graph 
    run_game(work_graph, attack_rate_list=attack_rate_list, defense_rate_list=defense_rate_list, random_steps_fn=random_steps)

    #test()

## This is the code that will be run if this method is run somewhere else

In [102]:
# Now run the game on work_graph 
def main():
    print(f'Running game from core file not in test block.')
    run_game(work_graph, attack_rate_list=attack_rate_list, defense_rate_list=defense_rate_list, random_steps_fn=random_steps)

#### Attempts for MGSS implementation (Not part of main code,  can be ignored)

In [103]:
def solve_stage(Ui, v_prev=None, tol=0.0):
    """
    Solve one stage of the lexicographic optimization LP as done in mgss.

    Ui: n-by-m numpy array (n = number of defender actions, m = number of attacker actions)
    v_prev: previously found v (float) for adding lexicographic constraints
    tol: tolerance for lexicographic constraints
    """

    n, m = Ui.shape

    # Variables: [v, x_1, x_2, ..., x_n]
    # Objective: minimize v => c = [1, 0, 0, ..., 0]
    c = np.zeros(n+1)
    c[0] = 1.0

    # Inequalities for each attacker route j:
    # -v + sum_i(Ui[i,j]*x_i) ≤ 0
    A_ub = np.zeros((m, n+1))
    b_ub = np.zeros(m)
    for j in range(m):
        A_ub[j,0] = -1.0
        for i in range(n):
            A_ub[j,i+1] = Ui[i,j]

    # Equality: sum_i x_i = 1
    A_eq = np.zeros((1, n+1))
    A_eq[0,1:] = 1.0
    b_eq = np.array([1.0])

    # If we have a previous v to enforce lexicographic constraints:
    # For example, v ≤ v_prev + tol
    if v_prev is not None:
        A_ub = np.vstack([A_ub, np.zeros((1,n+1))])
        A_ub[-1,0] = 1.0  # coefficient for v
        b_ub = np.append(b_ub, v_prev + tol)

    # Bounds: v >= 0, x_i >= 0
    bounds = [(0,None)]*(n+1)

    # Solve LP
    res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')
    if not res.success:
        raise ValueError("LP solver failed in solve_stage: " + res.message)

    v_opt = res.x[0]
    x_def = res.x[1:]

    return v_opt, x_def

print("Step 2: solve_stage function defined.")


Step 2: solve_stage function defined.


In [104]:
def lexicographic_optimization(U_matrices, tol=0.0):
    """
    Perform lexicographic optimization over multiple stages.
    
    U_matrices: list of arrays [U1, U2, ..., Uo]
        Each Ui is n-by-m and represents one stage of the lexicographic problem.
    tol: tolerance for lexicographic constraints
    
    Returns:
      v_values: list of v_opt for each stage
      x_def: x_def from the last stage
    """
    v_values = []
    x_def_final = None
    v_prev = None

    for p, Ui in enumerate(U_matrices, start=1):
        # Solve this stage:
        v_opt, x_def = solve_stage(Ui, v_prev=v_prev, tol=tol)
        v_values.append(v_opt)
        x_def_final = x_def
        
        # Update v_prev for the next stage:
        v_prev = v_opt

    return v_values, x_def_final

print("Step 3: lexicographic_optimization function defined.")


Step 3: lexicographic_optimization function defined.


In [105]:
def attacker_best_response(U_matrices_for_attacker, v_values, tol=0.0):
    """
    Compute the attacker's best response given the defender's strategy.
    
    U_matrices_for_attacker: list of arrays [U1', U2', ..., Uo'] for the attacker,
      each one is m-by-n or some configuration depending on how you define it.
      This could differ from defender's U because the roles are reversed (attacker chooses columns).
      
    v_values: the v_opt values found for the defender stages. These serve as reference to add lexicographic constraints for the attacker.
    tol: tolerance for lexicographic constraints.
    
    Returns:
      w_values: list of w_opt (attacker's analogous value) for each stage
      y_att: attacker's best response distribution from the last stage
    """
    # This is a placeholder function. In mgss, the attacker problem is 
    # solved in a similar lexicographic manner, but with different constraints:
    # Now we maximize something or minimize negative of something depending on how 
    # the attacker payoff is set up. In mgss, the attacker tries to achieve a certain lexicographic maximum.
    #
    # For demonstration, we will structure it similarly:
    #
    # Variables might be: w, y_j (probabilities of attacker routes)
    # Constraints: sum_j y_j = 1, y_j >= 0
    # and lexicographic constraints linking w to previously found v_values.
    
    # NOTE: Without a concrete definition of attacker problem, we just show a stub:
    
    w_values = []
    y_att_final = None
    w_prev = None
    
    for p, Ua in enumerate(U_matrices_for_attacker, start=1):
        # You would create and solve a similar LP here, just like solve_stage,
        # but from the attacker's perspective.
        #
        # For example:
        # maximize w
        # subject to constraints linking w and y_j to attacker payoffs
        # add lexicographic constraints w >= v_values[p-1] - tol or similar
        #
        # Since we don't have the actual attacker formulation here, we will just print a placeholder message:
        print(f"Attacker stage {p}: (Placeholder) Solve LP here based on Ua and v_values")
        
        # Placeholder solution:
        w_opt = v_values[p-1]  # Just a placeholder, in reality you'd solve another LP
        y_att = np.ones(Ua.shape[0]) / Ua.shape[0]  # a trivial uniform distribution as placeholder
        
        w_values.append(w_opt)
        y_att_final = y_att
        w_prev = w_opt
    
    return w_values, y_att_final

print("Step 4: attacker_best_response function (placeholder) defined.")


Step 4: attacker_best_response function (placeholder) defined.


In [57]:
# Example integration

# Suppose we have a defender with n=3 actions and an attacker with m=4 actions.
# We'll create two stages (o=2) of Ui matrices for demonstration.
np.random.seed(42)  # for reproducibility
Ui_stage1 = np.random.rand(3,4)  # random n-by-m matrix
Ui_stage2 = np.random.rand(3,4)  # another random stage

U_matrices = [Ui_stage1, Ui_stage2]

# Run lexicographic optimization for defender
v_values, x_def = lexicographic_optimization(U_matrices, tol=1e-5)
print("Defender lexicographic results:")
print("v_values:", v_values)
print("x_def (from last stage):", x_def)

# For attacker, we would have another set of matrices. Let's just make a placeholder:
U_matrices_for_attacker = [np.random.rand(4,3), np.random.rand(4,3)]  # assuming attacker chooses among 4 routes

# Compute attacker best response
w_values, y_att = attacker_best_response(U_matrices_for_attacker, v_values, tol=1e-5)
print("Attacker lexicographic results (placeholder):")
print("w_values:", w_values)
print("y_att (from last stage):", y_att)

print("Step 5: Integration example done.")


Defender lexicographic results:
v_values: [np.float64(0.6873214761782448), np.float64(0.4021425673969857)]
x_def (from last stage): [0.         0.68173949 0.31826051]
Attacker stage 1: (Placeholder) Solve LP here based on Ua and v_values
Attacker stage 2: (Placeholder) Solve LP here based on Ua and v_values
Attacker lexicographic results (placeholder):
w_values: [np.float64(0.6873214761782448), np.float64(0.4021425673969857)]
y_att (from last stage): [0.25 0.25 0.25 0.25]
Step 5: Integration example done.


#### MGSS Implementation End